# 使用DirectML进行GPU模型训练
> 我的RX480又可以跑模型了

- toc: true
- badges: true
- comments: true
- hide: false
- categories: [tensorflow, directml]

## 关于[DirectML](https://github.com/microsoft/DirectML)
详情看README。主要是支持所有“支持DX12的显卡”进行硬件加速运算。这对于手上只有A卡的我无疑又是个好东西。  

先前已经写过一篇关于[PlaidML](https://bjchacha.github.io/myblog/ai/2020/02/25/using-plaidml-keras-to-ai.html)的博文，也是可用A卡硬件加速。所以这里省略部分步骤，并最后跟PlaidML比较下效果。

## 安装DirectML
目前[Tensorflow with DirectML](https://docs.microsoft.com/zh-cn/windows/win32/direct3d12/gpu-tensorflow-windows)仅支持最新版本的Windows 10和WSL。  

安装非常简单，直接`pip`一下就好。这里例行使用`conda`创建虚拟环境来运行DirectML。

> conda create -n directml python=3.7  
> conda activate directml  
> pip install tensorflow-directml

> Note: DirectML只支持Tensorflow 1.15.

## 使用DirectML训练Fashion-MNIST分类器
这里跑跟之前试用PlaidML时一样的代码，方便对比。

In [1]:
#collapse
import tensorflow as tf
from time import time

data = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = data.load_data()

x_train = x_train.astype('float32').reshape(-1, 28, 28, 1) / 255.
x_test = x_test.astype('float32').reshape(-1, 28, 28, 1) / 255.
# print(x_train.shape)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28, 28, 1)
    ),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(
        filters=32, kernel_size=2, padding='same', activation='relu'
    ),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation='softmax')])
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy'])

train_start = time()
model.fit(x_train, y_train, batch_size=64, epochs=10)
train_end = time()
_, accuracy = model.evaluate(x_test, y_test)
print('training time cost: {0:.1f} s, accuracy: {1:.4f}'.format(train_end-train_start, accuracy))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 14s 226us/sample - loss: 0.5786 - acc: 0.7869
Epoch 2/10
60000/60000 [==============================] - 13s 214us/sample - loss: 0.4072 - acc: 0.8522
Epoch 3/10
60000/60000 [==============================] - 13s 213us/sample - loss: 0.3625 - acc: 0.8681
Epoch 4/10
60000/60000 [==============================] - 13s 216us/sample - loss: 0.3340 - acc: 0.8788
Epoch 5/10
60000/60000 [==============================] - 13s 213us/sample - loss: 0.3179 - acc: 0.8842
Epoch 6/10
60000/60000 [==============================] - 13s 218us/sample - loss: 0.3003 - acc: 0.8903
Epoch 7/10
60000/60000 [==============================] - 13s 213us/sample - loss: 0.2891 - acc: 0.8933
Epoch 8/10
60000/60000 [==============================] - 13s 213us/sample - loss: 0.2768 - acc: 0.8980
Epoch 9/10
60000/60000 [==============================] - 13s 21

上面可以看到DirectML可以正常使用A卡进行训练，训练时长为129秒，而PlaidML跑了124秒。  
虽说DirectML比PLaidML慢，但胜在支持所有DX12的显卡以及完整的Tensorflow（PlaidML只支持Keras）。